# Instrucciones.

Debería poder realizar todo el laboratorio en 3 horas.

**Recomendamos fuertemente** que siga trabajando en el laboratorio durante las horas asignadas, pues así puede consultar en vivo dudas que tenga sobre las partes finales del mismo. 

* Complete la celda "A. Datos grupo".
* Descargue los archivos de instancias de ucursos.
* Complete la sección "B. Preparación y definición del problema" (incluye 1 ejercicio)
* Envie el archivo .ipynb por el modulo de tareas de ucursos en la TAREA: Laboratorio 3 presencial 
* Plazo de entrega:  Miércoles 3 de junio a las 18:00

La parte B es muy corta (es aproximadamente 1/5 del laboratorio), guarde el archivo apenas la complete para poder subirlo a ucursos.

La parte C del laboratorio se  entrega el día Viernes a las 22:00

* Complete todos los ejercicios
* Envie el archivo .ipynb por el modulo de tareas de ucursos en la TAREA: Laboratorio 3 completo
* Plazo de entrega: Viernes 5 de mayo a las 22:00 
  (puede solicitar por correo electrónico plazo adicional si lo necesita, pero su solicitud debe hacerse antes del viernes  a las 22:00. No es automático)


# A. Datos grupo

Escriba en esta celda el nombre de cada integrante de su grupo.

- Integrante 1: XXX

- Integrante 2: YYY

 # B. Preparación y definición del problema.

1. Usaremos los paquetes `JuMP, Gurobi, CSV, StatsPlots, LinearAlgebra, SparseArrays`
   Asegúrese de tener la última versión de `JuMP` corriendo (0.21.2 en mayo 2020)
   Si le falta algún paquete instálelo (`using Pkg`; `Pkg.add("nombre_paquete")`) o actualícelo (`Pkg.update("nombre_paquete")`)
 

In [ ]:
# using Pkg
# Pkg.add(" ")
# Pkg.update(" ")

using JuMP, Gurobi,CSV, StatsPlots, SparseArrays, LinearAlgebra

┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1273


2. Al igual que en el laboratorio anterior, cree una constante `GUROBI_ENV` que guarde el ambiente gurobi que usaremos
  (de modo de evitar múltiples instancias). Además incluya el archivo `cutting.jl` que contiene funciones útiles para 
  leer archivos y graficar respuestas


In [ ]:
const GUROBI_ENV = Gurobi.Env()
include("cutting.jl")

3. Hoy trabajaremos con el **Cutting Stock problem** explicado en clases: Una papelera dispone de rollos padres (troncos) de largo $W>0$.
  Recibe $m$ pedidos. Cada pedido $i$ consiste en una demanda $b_i$ de rollos de ancho $w_i$ 
  La papelera debe luego cortar cada tronco en rollos de los largos adecuados para satisfacer los pedidos.
  El objetivo del **Cutting Stock problem** es determinar el mínimo número de troncos necesarios para satisfacer los pedidos.
  
  En clases escribimos la formulación (CS1) para el problema escrito abajo. El entero $N$ representa una cota superior inicial (ojalá pequeña) que permite satisfacer las $m$ demandas.

\begin{align*}
(\text{CS1})\qquad \min \sum_{j=1}^N y_j&\\
\sum_{j=1}^N z_{i,j}&= b_i,& \forall i\in [m]\\
\sum_{i=1}^m z_{i,j}w_{i}&\leq Wy_j,&\forall j\in [N]\\
z&\in \mathbb{N}^{[m]\times [N]}\\
y&\in \{0,1\}^{[N]}
\end{align*}

## Ejercicio 1:

*  Los archivos "cuttingstock01.csv","cuttingstock02.csv","cuttingstock03.csv","cuttingstock04.csv", "cuttingstock05.csv" contienen instancias de prueba para el problema de CuttingStock en formato csv (comma-separated-values)

  Puede leer una instancia puede leerla usando la siguiente función
  ```julia
  anchotronco, ancho, demanda = lee_cuttingstock(nombre_archivo_csv)
  ```
  Esta función guardará en `anchotronco`  el ancho de los rollos padres (un entero), en `ancho` y `demanda` vectores con los anchos y demandas de cada pedido. 


*  El archivo "visualizacion.jl" contiene funciones para visualizar salidas del problema. En este ejercicco 
  la función "grafica" es relevante.
  Sintaxis: 
  ```julia
  grafica(ancho, yvalues, zvalues)
  ```
  donde `ancho` es un vector de anchos, e `yvalues`, `zvalues` son soluciones factibles del modelo (CS1)

Este ejercicio tiene 3 partes 

a) Escribir una función ModeloSimple que cree el modelo CS1.

b) Escribir una función ResolverSimple que automatice el proceso de: 
   leer-entrada, crear modelo, optimizar y graficar salida.
   
c) Ejecutar la función ResolverSimple en dos instancias de prueba.


**Importante**: Para obligar al solver a devolver una solución incluso si fija `cotatiempo` a ser muy bajo, aprovecharemos de entregarle un incumbente o solución factible de partida. Esto se conoce como **warm-start**.

La siguiente función devuelve la solución factible trivial que consiste en asignar un tronco completo por cada unidad pedida por cada cliente.

In [ ]:
function solucioninicial(anchotronco, ancho, demanda)
    m=size(ancho,1)
    N=sum(demanda)
    yinicial= ones(Int8, N)
    zinicial = zeros(Int8,m,N)
    aux=0
    for i in 1:m
        for j in 1:demanda[i]
            zinicial[i,j+aux]=1
        end
        aux+=demanda[i]
    end
    return m, N, yinicial, zinicial 
end

### 1.a)

Escriba una función ModeloSimple que siga la siguiente sintaxis de entrada y salida:
```julia
function ModeloSimple(anchotronco, ancho, demanda, cotatiempo=30)
return modeloCS1
```
donde la salida es

* `modelo` es un objeto de tipo Model (creado por JuMP) basado en la formulación (CS1).

y la entrada es

* `anchotronco`:  Ancho de los troncos
* `ancho`: Vector de ancho de pedidos
* `demanda`: Vector de demanda de pedidos
* `cotatiempo`: Cota máxima en tiempo (en segundos) que se le permite al solver utilizar. (Al escribir cotatiempo=30 en la declaración esto se transforma en un parámetro opcional)


In [ ]:
function ModeloSimple(anchotronco, ancho, demanda, cotatiempo=30)
    #supone variable GUROBI_ENV global
    
    # Paso 1 Recuperar parametros y solucion inicial
    m, N, yinicial, zinicial = solucioninicial(anchotronco, ancho, demanda)
   
    # Paso 2, crear el modelo. Aqui se usa el parámetro cotatiempo
    modeloCS1 = Model(optimizer_with_attributes(
                () -> Gurobi.Optimizer(GUROBI_ENV), 
                "OutputFlag" => 1,
                "TimeLimit" => cotatiempo)) 
    
    # Paso 3 cree las variables, restricciones y objetivos del modelo
    @variable(..)
    @objective(..)
    @constraints(..)
    
    # Paso 4, fije la solucion inicial (esto supone que llamo a sus variables y, z)
    set_start_value.(y, yinicial)
    set_start_value.(z, zinicial)
    
    return modeloCS1
end

1.b) 
Escriba una función ResolverSimple que siga la siguiente sintaxis de entrada y salida:

```julia
function ResolverSimple(nombre_archivo_csv, cotatiempo=30)
    #debe graficar en pantalla la solución
    return valor_objetivo, yvalues, zvalues, mejor_cota
```

aquí `valor_objetivo`, `yvalues` y `zvalues` son el objetivo y los mejores vectores soluciones encontrados para (CS1) -- (podrían ser no-óptimos por cota de tiempo) y `mejor_cota` es la mejor cota inferior encontrada para la solución de (CS1)

Para escribir la función puede usar la siguiente plantilla.

In [ ]:
function ResolverSimple(nombre_archivo_csv, cotatiempo=30)
    # cargar instancia
    anchotronco, ancho, demanda = lee_cuttingstock(nombre_archivo_csv)

    # crear modelo
    modelo1 = ..
    
    # optimizar, extraer objetivos y valores
    
    optimize!(modelo1)
    if termination_status(modelo1) == MOI.TIME_LIMIT
        @warn("Límite de tiempo. solución suboptima")
    end
    
    valor_objetivo = ..
    mejor_cota = objective_bound(modelo1)
    yvalues = values.(modelo1[:nombrevariable])
    zvalues = ..

    # grafique
    grafica(...)
    
    # entregue
    return valor_objetivo, yvalues, zvalues, mejor_cota
    
end

### 1.c) 
Ejecute la función ResolverSimple en las instancia de pruebas "cuttingstock01.csv" y "cuttingstock02.csv" 
con limites de 30 segundos cada uno.

En el último ejercicio se le pedirá volver a usar estas funciones para comparar sus resultados con los de generación de columnas.

In [ ]:
ResolverSimple()

In [ ]:
ResolverSimple()

#  C. Generación de columnas 

Resumimos abajo la manera de solucionar este problema mediante generación de columna. 

El modelo completo por patrones es

$$
\begin{align*}
(\text{CS}2)\qquad \min \sum_{p \in P} x_p &\\
\sum_{p\in P} x_p p_{i}&= b_i,& \forall i\in [m]\\
x_p&\in \mathbb{N}, &\forall p \in P
\end{align*}
$$

Donde $P$ es un conjunto de patrones, siendo un patrón un vector de $m$ filas y corresponde a una forma factible de cortar un tronco, así para $p\in P$ se tiene que $p_{i}$ con $i \in [m]$ corresponde al número de items del pedido $i$ que posee el patrón $p$.

Generación de columnas permite resolver la relajación fraccional (PL) de (CS2) en la que $x_p \in \mathbb{N}$ se reemplaza por $x_p\geq 0$.

Para esto necesitamos saber escribir el master y el pricing problem asociados:

**Master Problem**


Sea $Q^{k}$ el conjunto de patrónes al comienzo de la iteración $k+1$, con esto el master problem (fraccional):

\begin{align*}
(\text{MP}(Q^{k}))\qquad \min \sum_{p\in Q^{k}} x_{p}&\\
\sum_{p\in Q^{k}} x_{p}p_{i}&= b_{i},& \forall i\in [m]\\
x_{p}&\geq 0 & \forall p \in Q^{k}
\end{align*}


**Pricing Problem**

De clases, sabemos que si $y$ es una solución del dual del master problem, entonces el pricing problem de $y$ consiste en determinar un vector $p$ de valor $\eta'$ dado por el siguiente knapsack problem:

\begin{align*} 
(\text{SP}(y))\qquad \max \sum_{i=1}^{m}y_{i}p_{i} &\\
\sum_{i=1}^m p_{i}w_{i}&\leq W\\
p &\in \mathbb{N}^{[m]}\\
\end{align*}

**Generación de columna**

El algoritmo de generación de columna consiste en lo siguiente:

1. Elegir conjunto de columnas inicial ($Q^{0}$) que guardaremos como columnas de la matriz `patron`.

   Originalmente, esta matriz será la identidad (tendrá $m$ columnas y la $j$-ésima columna solo tendrá un 1 
   en la coordenada $j$). Dado que durante el algoritmo, la matriz tendrá mucho ceros se recomienda guardarla como un 
   arreglo sparse. Puede inicializar una matriz sparse igual a la identidad usando el comando
   ```julia
   patron = sparse(I,m,m)
   ```
   donde $I$ es la matriz identidad (viene por defecto al cargar el paquete LinearAlgebra), y $m$ es el número de 
   pedidos de la instancia.
   
   
2. Repetir:

   Calcular el óptimo y dual óptimo $(x,y)$ del master problem actual `MP(patron)`
   
   Calcular el valor $\eta'$ y el patrón óptimo $p$ del pricing problem `Pricing(y)`
   
   Si $\eta'\leq 1$ terminar. El vector $x$ es óptimo.
   
   Si $\eta'\geq 1$ agregar $p$ al conjunto de patrones actual. En Julia esto se puede hacer concatenando $p$ al final de la matriz patron, escribiendo
   ```julia
   patron = hcat(patron, nuevacolumna)
   ```
   
En los siguientes ejercicios deberá crear una función que resuelva el problema mediante generación de columnas.    

### Ejercicio 2:

Escriba una función `MasterProblem` que crea y resuelva el modelo asociado al problema maestro. 
La función debe tener el siguiente formato

```julia
function MasterProblem(demanda, patron)
return valor_objetivo, xvalues, yvalues
```
donde la salida es

* `valor_objetivo` es el objetivo del problema primal
* `xvalues` es el vector x óptimo encontrado
* `yvalues` es el vector y dual-óptimo encontrado

y la entrada es

* `demanda`: Vector de demanda de pedidos
* `patron`: matriz con $m$ filas. En su $j$-ésima columna tiene el $j$-ésimo patrón.

*Observación importante:*
  Si en la declaración de restricciones les pone nombre, por ejemplo
  
  ```julia
  @constraints(mimodelo, nombre_restriccion[i=1:m], restriccion)
  ```
  y luego optimiza
  puede recuperar el vector de variables duales optima asociada escribiendo
  ```julia
  dual.(nombre_restriccion)
  ```
  
  Puede usar y completar la siguiente plantilla

In [ ]:
function MasterProblem(demanda, patron)
    
    # Paso 1, Recuperar parámetros
    m =  # número de pedidos
    ncols =  # número de patrones
    
    # Paso 2, crear modelo.
    modelomaestro = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"OutputFlag" => 0))
    
    # Paso 3 cree las variables, restricciones y objetivos del modelo
    ...
    
    # Paso 4, optimizar modelo
    optimize!(modelomaestro)
    if termination_status(modelomaestro) != MOI.OPTIMAL
        @warn("No hay optimo")
    end
    
    # Paso 5, obtener la salida
    ...
    
    return valor_objetivo, xvalues, yvalues
end

### Ejercicio 3:

Escriba una función `PricingProblem` que crea y resuelva el modelo asociado al pricing problema. La función debe tener el siguiente formato

```julia
function PricingProblem(anchotronco, ancho, duales)
return etaprima, patronoptimo
```
donde la salida es

* `etaprima` es el objetivo del problema de pricing
* `patronoptimo` es el vector $p$ óptimo encontrado

y la entrada es

* `anchotronco`:  Ancho de los troncos
* `ancho`: Vector de ancho de pedidos
* `duales`: Vector $y$ dual


  Puede usar y completar la siguiente plantilla


In [ ]:
function PricingProblem(anchotronco, ancho, duales)
    # Paso 1 Recuperar parametros
    m = ...
    # Paso 2, crear modelo.
    modelopricing = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"OutputFlag" => 0))
    
    # Paso 3 cree las variables, restricciones y objetivos del modelo
    ...
    
    # Paso 4, optimizar modelo
    optimize!(modelopricing)
    if termination_status(modelopricing) != MOI.OPTIMAL
        @warn("No hay optimo")
    end
    
    # Paso 5, obtener la salida
    ...
       
    return etaprima, patronoptimo
end

### Ejercicio 4:

Ahora implementaremos el método de generación de columnas siguiendo las indicaciones para ello.
Para evitar tomar demasiado tiempo, usaremos un parámetro adicional llamado `maxcols` que limita el número de columnas **adicionales** creadas. Además debe probar su método en las instancias  "cuttingstock01.csv" y "cuttingstock02.csv" 

### 4.1 Implemente el método

```julia
function generacioncolumnas(anchotronco, ancho, demanda, maxcols)
    return mensaje, patron, xvalues, valor_objetivo
```
donde la salida es

* `mensaje` puede ser "optimo" si el proceso terminó, o "suboptimo" si se alcanzó el limite de columnas antes de terminar.
* `patron` es la matriz que tiene en sus columnas los patrones usados.
* `xvalues` es el vector x encontrado.
* `valor_objetivo` es el valor de la solución encontrada.

y la entrada es

* `anchotronco`:  Ancho de los troncos
* `ancho`: Vector de ancho de pedidos
* `demanda`: Vector de demanda de pedidos
* `maxcols`: Número máximo de columnas adicionales que se crearán

  Puede usar y completar la siguiente plantilla


In [ ]:
function generacioncolumnas(anchotronco, ancho, demanda, maxcols)
    # Definir m y conjunto inicial de patrones (ver indicaciones al principio de sección C)
    m = size(demanda,1)
    patron = sparse(I,m,m)
    ncols = m # numero inicial de columnas
    
    #resolver una vez fuera del while para tener inicializados los vectores apropiados 
    valor_objetivo, xvalues, yvalues = MasterProblem(demanda, patron) 
    # Repetir 
         while ncols <= m + maxcols 
        # Resolver problema maestro
        .. = MasterProblem(..)
        
        # Resolver pricing problem asociado a solucion dual
        .. = PricingProblem(..)
        
        # Reportar en pantalla valor de etaprima
        println("Iteración : ", ncols - m + 1, ", η'=", etaprima)
        
        # Si etaprima<=1 terminar
        if etaprima<=1
            println("optimo", )
            return ...
        end
        
        # Si no, concatenar columna encontrada a patrones y aumentar numero de columnas
        patron = ...
        ncols += 1     
    end
    
    # Fuera de while: se alcanzó el número de iteraciones
    println("suboptimo", )
    return ...
end

### 4.2 Pruebe su método en instancias "cuttingstock01.csv" y "cuttingstock02.csv"  y grafique su salida

Para graficar use la función

```julia
graficapatrones(ancho, patron, xvalues)
``` 
incluida en el archivo visualizacion.jl


In [ ]:
# cargar instancia
anchotronco1, ancho1, demanda1 = lee_cuttingstock("cuttingstock01.csv")
# ejecute
mensaje1, patron1, xvalues1, valor_objetivo1 = generacioncolumnas(anchotronco1, ancho1, demanda1, 50)
# grafique
graficapatrones(ancho1, patron1, xvalues1)


In [ ]:
# cargar instancia
anchotronco2, ancho2, demanda2 = lee_cuttingstock("cuttingstock02.csv")
# ejecute
mensaje2, patron2, xvalues2, valor_objetivo2 = generacioncolumnas(anchotronco2, ancho2, demanda2, 50)
# grafique
graficapatrones(ancho2, patron2, xvalues2)

### Ejercicio 5. 

El ejercicio anterior permite encontrar soluciones fraccionales para (PL). Para poder transformar estas soluciones en soluciones enteras factibles -- no necesariamente óptimas -- para (CS2) se puede resolver la versión **integral** del master problem con el conjunto de patrones finales encontrado.

Para esto, cree una función 
```julia
function MaestroIntegral(demanda, patron, cotatiempo)
return valor_objetivo, xvalues
```
que haga lo mismo que `Maestro` pero que imponga integralidad en el vector $x$ del modelo (ojo, no debe calcular los valores duales), agregue una cotatiempo en segundos para su resolución.

Finalmente aplique `MaestroIntegral` a los patrones encontrados en el ejercicio anterior para las instancias "cuttingstock01.csv" y "cuttingstock02.csv" y grafíquelos.


In [ ]:
function MaestroIntegral(demanda, patron, cotatiempo)
    ...
    return valor_objetivo, xvalues
end

In [ ]:
#usando resultados de antes, encuentre factible integral instancia 1
valor_objetivo_integral1, xvalues_integral1 = MaestroIntegral(demanda1, patron1)

In [ ]:
graficapatrones(ancho1, patron1, xvalues_integral1)

In [ ]:
#usando resultados de antes, encuentre factible integral instancia 2
valor_objetivo_integral2, xvalues_integral2 = MaestroIntegral(demanda2, patron2)

In [ ]:
graficapatrones(ancho2, patron2, xvalues_integral2)

### Ejercicio 6. 

En este laboratorio usted ha encontrado soluciones y cotas para el problema de cutting stock en distintas instancias
Ejecute las funciones y comandos apropiadas nuevamente en las siguientes celdas (el archivo que entregue debe incluir su ejecución) para llenar (a mano si lo desea) la siguiente tabla, usando **150** columnas adicionales para resolver (LP) y (CS2) **60** segundos como maximo en la ejecución de (CS1) y (CS2).


| Instancia      | mejor-solucion-(CS1) | mejor-cota-(CS1) | mejor-solucion-(LP) | mejor-solucion-(CS2) |
|----------------|----------------------|------------------|---------------------|----------------------|
| cuttingstock01 | valor01              | valor01          | valor01             | valor01              |
| cuttingstock02 | valor02              | valor02          | valor02             | valor02              |
| cuttingstock03 | valor03              | valor03          | valor03             | valor03              |
| cuttingstock04 | valor04              | valor04          | valor04             | valor04              |

**OPCIONAL:**
¿Puede explicar la relación entre estos números?
(cuidado! recuerde que mejor-solucion-(LP) podria no ser optima para (LP) ¿que pasa en ese caso?)

In [ ]:
# Use una o más celdas para mostrar como obtuvo los valores de la tabla.

In [ ]:
# Use una o más celdas para mostrar como obtuvo los valores de la tabla.

### Ejercicio 7 (opcional)

Usando los metodos de este laboratorio (o modificándolos como lo desee) encuentre la mejor solución entera y la mejor cota inferior que pueda para las 4 instancias y para la instancia **cuttingstock05.csv** (que es muchísimo más grande).

¿Puede encontrar soluciones óptimas en todas ellas?

* HINT: Para la instancia grande le sugerimos buscar una mejor solucion inicial en (CS1). 
  Además, comente todo lo que no sea necesario (como graficar)

* Ideas: * 
  Recuerde que la formulación (PL) es siempre cota inferior del óptimo.
  Puede usar (CS1) para extraer patrones nuevos que agregar a los patrones encontrados para la solución factible de (CS2)
  También puede usar la solución encontrada de (CS2) para alimentar (CS1) como incumbente inicial.
